# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 4 2023 8:30AM,254207,10,8985344,Livs Products,Released
1,Jan 3 2023 3:09PM,254208,19,ADV80009476,"AdvaGen Pharma, Ltd",Released
2,Jan 3 2023 2:57PM,254206,19,ADV80010014,"AdvaGen Pharma, Ltd",Released
3,Jan 3 2023 2:57PM,254206,19,ADV80010015,"AdvaGen Pharma, Ltd",Released
4,Jan 3 2023 2:57PM,254206,19,ADV80010016,"AdvaGen Pharma, Ltd",Released
5,Jan 3 2023 2:57PM,254206,19,ADV80010017,"AdvaGen Pharma, Ltd",Released
6,Jan 3 2023 2:57PM,254206,19,ADV80010018,"AdvaGen Pharma, Ltd",Released
7,Jan 3 2023 2:57PM,254206,19,ADV80009959,"AdvaGen Pharma, Ltd",Released
8,Jan 3 2023 2:57PM,254206,19,ADV80009960,"AdvaGen Pharma, Ltd",Released
9,Jan 3 2023 2:57PM,254206,19,ADV80009961,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
48,254204,Released,5
49,254205,Released,8
50,254206,Released,17
51,254207,Released,1
52,254208,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254204,NaN,NaN,5.0
254205,NaN,NaN,8.0
254206,NaN,NaN,17.0
254207,NaN,NaN,1.0
254208,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254090,55.0,6.0,1.0
254104,0.0,0.0,1.0
254108,0.0,1.0,23.0
254111,0.0,53.0,0.0
254117,0.0,37.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254090,55,6,1
254104,0,0,1
254108,0,1,23
254111,0,53,0
254117,0,37,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254090,55,6,1
1,254104,0,0,1
2,254108,0,1,23
3,254111,0,53,0
4,254117,0,37,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254090,55,6,1
1,254104,,,1
2,254108,,1,23
3,254111,,53,
4,254117,,37,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 4 2023 8:30AM,254207,10,Livs Products
1,Jan 3 2023 3:09PM,254208,19,"AdvaGen Pharma, Ltd"
2,Jan 3 2023 2:57PM,254206,19,"AdvaGen Pharma, Ltd"
19,Jan 3 2023 2:56PM,254205,10,MedStone Pharma LLC
27,Jan 3 2023 2:54PM,254204,10,"Nextsource Biotechnology, LLC"
32,Jan 3 2023 2:53PM,254203,19,"AdvaGen Pharma, Ltd"
58,Jan 3 2023 2:42PM,254201,10,Ivaoes Animal Health
59,Jan 3 2023 2:36PM,254199,19,"AdvaGen Pharma, Ltd"
60,Jan 3 2023 2:35PM,254198,19,"AdvaGen Pharma, Ltd"
86,Jan 3 2023 1:59PM,254195,15,"Virtus Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 4 2023 8:30AM,254207,10,Livs Products,,,1
1,Jan 3 2023 3:09PM,254208,19,"AdvaGen Pharma, Ltd",,,1
2,Jan 3 2023 2:57PM,254206,19,"AdvaGen Pharma, Ltd",,,17
3,Jan 3 2023 2:56PM,254205,10,MedStone Pharma LLC,,,8
4,Jan 3 2023 2:54PM,254204,10,"Nextsource Biotechnology, LLC",,,5
5,Jan 3 2023 2:53PM,254203,19,"AdvaGen Pharma, Ltd",,,26
6,Jan 3 2023 2:42PM,254201,10,Ivaoes Animal Health,,,1
7,Jan 3 2023 2:36PM,254199,19,"AdvaGen Pharma, Ltd",,,1
8,Jan 3 2023 2:35PM,254198,19,"AdvaGen Pharma, Ltd",,,26
9,Jan 3 2023 1:59PM,254195,15,"Virtus Pharmaceuticals, LLC",,,11


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 4 2023 8:30AM,254207,10,Livs Products,1,,
1,Jan 3 2023 3:09PM,254208,19,"AdvaGen Pharma, Ltd",1,,
2,Jan 3 2023 2:57PM,254206,19,"AdvaGen Pharma, Ltd",17,,
3,Jan 3 2023 2:56PM,254205,10,MedStone Pharma LLC,8,,
4,Jan 3 2023 2:54PM,254204,10,"Nextsource Biotechnology, LLC",5,,
5,Jan 3 2023 2:53PM,254203,19,"AdvaGen Pharma, Ltd",26,,
6,Jan 3 2023 2:42PM,254201,10,Ivaoes Animal Health,1,,
7,Jan 3 2023 2:36PM,254199,19,"AdvaGen Pharma, Ltd",1,,
8,Jan 3 2023 2:35PM,254198,19,"AdvaGen Pharma, Ltd",26,,
9,Jan 3 2023 1:59PM,254195,15,"Virtus Pharmaceuticals, LLC",11,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 4 2023 8:30AM,254207,10,Livs Products,1,,
1,Jan 3 2023 3:09PM,254208,19,"AdvaGen Pharma, Ltd",1,,
2,Jan 3 2023 2:57PM,254206,19,"AdvaGen Pharma, Ltd",17,,
3,Jan 3 2023 2:56PM,254205,10,MedStone Pharma LLC,8,,
4,Jan 3 2023 2:54PM,254204,10,"Nextsource Biotechnology, LLC",5,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 4 2023 8:30AM,254207,10,Livs Products,1.0,NaN,NaN
1,Jan 3 2023 3:09PM,254208,19,"AdvaGen Pharma, Ltd",1.0,NaN,NaN
2,Jan 3 2023 2:57PM,254206,19,"AdvaGen Pharma, Ltd",17.0,NaN,NaN
3,Jan 3 2023 2:56PM,254205,10,MedStone Pharma LLC,8.0,NaN,NaN
4,Jan 3 2023 2:54PM,254204,10,"Nextsource Biotechnology, LLC",5.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 4 2023 8:30AM,254207,10,Livs Products,1.0,0.0,0.0
1,Jan 3 2023 3:09PM,254208,19,"AdvaGen Pharma, Ltd",1.0,0.0,0.0
2,Jan 3 2023 2:57PM,254206,19,"AdvaGen Pharma, Ltd",17.0,0.0,0.0
3,Jan 3 2023 2:56PM,254205,10,MedStone Pharma LLC,8.0,0.0,0.0
4,Jan 3 2023 2:54PM,254204,10,"Nextsource Biotechnology, LLC",5.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,7116487,493.0,123.0,2.0
12,508302,0.0,21.0,0.0
15,1016697,124.0,2.0,0.0
19,2287649,77.0,9.0,55.0
20,508253,2.0,0.0,0.0
21,254144,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,7116487,493.0,123.0,2.0
1,12,508302,0.0,21.0,0.0
2,15,1016697,124.0,2.0,0.0
3,19,2287649,77.0,9.0,55.0
4,20,508253,2.0,0.0,0.0
5,21,254144,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,493.0,123.0,2.0
1,12,0.0,21.0,0.0
2,15,124.0,2.0,0.0
3,19,77.0,9.0,55.0
4,20,2.0,0.0,0.0
5,21,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,493.0
1,12,Released,0.0
2,15,Released,124.0
3,19,Released,77.0
4,20,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21
Status,,,,,,
Completed,2.0,0.0,0.0,55.0,0.0,0.0
Executing,123.0,21.0,2.0,9.0,0.0,1.0
Released,493.0,0.0,124.0,77.0,2.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21
0,Completed,2.0,0.0,0.0,55.0,0.0,0.0
1,Executing,123.0,21.0,2.0,9.0,0.0,1.0
2,Released,493.0,0.0,124.0,77.0,2.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21
0,Completed,2.0,0.0,0.0,55.0,0.0,0.0
1,Executing,123.0,21.0,2.0,9.0,0.0,1.0
2,Released,493.0,0.0,124.0,77.0,2.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()